In [1]:
%matplotlib inline

In [2]:
import sys
import pickle
from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from networkx.algorithms.approximation import clique
from sklearn.metrics import accuracy_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import Ridge
from sklearn.svm import SVC, SVR
from tqdm.notebook import tqdm

from helpers import SBM_Data, load_or_calc_and_save, ytrue_to_partition, calc_avranks, RFE, OneVsRest_custom

sys.path.append('../../pygkernels')
from pygkernels.scenario import d3_category20

In [3]:
data_hub = SBM_Data()

X, y = data_hub.make_dataset()
X_train, y_train = X[:, :70].reshape(-1, X.shape[2]), y[:, :70].reshape(-1, y.shape[2])
X_val,   y_val   = X[:, 70:].reshape(-1, X.shape[2]), y[:, 70:].reshape(-1, y.shape[2])
X,       y       = X.reshape(-1, X.shape[2]), y.reshape(-1, y.shape[2])

_, y_ari = data_hub.make_dataset(return_clf=False)
ari_train = y_ari[:, :70].reshape(-1, y_ari.shape[2])
ari_val = y_ari[:, 70:].reshape(-1, y_ari.shape[2])

feature_names = data_hub.allowed_features_list

prepare columns:   0%|          | 0/7 [00:00<?, ?it/s]

wrapper: RECALC ../../cache/cache/feature_importance/dataset2sbm_polbooks.pkl. args: , kwargs: 


FileNotFoundError: [Errno 2] No such file or directory: '../../cache/kkmeans_init_sbm/graphs/dataset2sbm_polbooks_100_graphs.pkl'

# Feature importance

In [ ]:
X.shape

In [ ]:
fig, ax = plt.subplots(5, 5, figsize=(30, 30))
ax = ax.ravel()
for i, name in enumerate(data_hub.allowed_features_list):
    ax[i].hist(X[:, i], bins=20)
    ax[i].set_title(name)

# Remove useless measures

In [ ]:
plot_useless = []
support_kernels = np.array([True] * 25)
while np.sum(support_kernels) > 0:
    ari_subset = ari_train[:, support_kernels]
    kernel_names_subset = np.array(data_hub.kernel_names)[support_kernels]
    most_useless = ('none', ari_subset.shape[0] + 1)
    for idx, kernel_name in enumerate(kernel_names_subset):
        a = np.array([True] * np.sum(support_kernels))
        a[idx] = False
        y_measure = ari_subset[:, idx]
        y_rest = np.max(ari_subset[:, a], axis=1) if np.sum(a) > 0 else -1
        stat = Counter(y_measure > (y_rest + 0.001))
        if stat[True] < most_useless[1]:
            most_useless = (kernel_name, stat[True])
    support_kernels[data_hub.kernel_names.index(most_useless[0])] = 0
    maxari = np.mean(np.max(ari_train[:, support_kernels], axis=1)) if np.sum(support_kernels) > 0 else 0
    most_useless += (maxari, )
    print(f'{np.sum(support_kernels)}/25: most useless={most_useless[0]}, best in {most_useless[1]}/{ari_subset.shape[0]}, '
          f'max quality wo this measure {maxari:.4f}')
    plot_useless.append(most_useless)

plt.plot([x[1]/ari_subset.shape[0] for x in plot_useless], label='best in share')
plt.plot([x[2] for x in plot_useless], label='ARI')
plt.legend()

In [ ]:
support_kernels = np.array([True] * 25)
while np.sum(support_kernels) > 14:
    ari_subset = ari_train[:, support_kernels]
    kernel_names_subset = np.array(data_hub.kernel_names)[support_kernels]
    most_useless = ('none', ari_subset.shape[0] + 1)
    for idx, kernel_name in enumerate(kernel_names_subset):
        a = np.array([True] * np.sum(support_kernels))
        a[idx] = False
        y_measure = ari_subset[:, idx]
        y_rest = np.max(ari_subset[:, a], axis=1) if np.sum(a) > 0 else -1
        stat = Counter(y_measure > (y_rest + 0.001))
        if stat[True] < most_useless[1]:
            most_useless = (kernel_name, stat[True])
    support_kernels[data_hub.kernel_names.index(most_useless[0])] = 0
    maxari = np.mean(np.max(ari_train[:, support_kernels], axis=1)) if np.sum(support_kernels) > 0 else 0
    most_useless += (maxari, )
    print(f'{np.sum(support_kernels)}/25: most useless={most_useless[0]}, best in {most_useless[1]}/{ari_subset.shape[0]}, '
          f'max quality wo this measure {maxari:.4f}')
    plot_useless.append(most_useless)
    
print(support_kernels.__repr__())
    
suport_kernels_names = np.array(data_hub.kernel_names)[support_kernels]
print(suport_kernels_names.__repr__())

In [ ]:
y_train = y_train[:, support_kernels]
y_val = y_val[:, support_kernels]

# One-vs-rest SVC

In [ ]:
# estimator = OneVsRestClassifier(SVC())
estimator = OneVsRest_custom(SVR(), weight_samples=True)
selector = RFE(estimator, feature_names, max_features=4, n_jobs=12)
selector = selector.fit(X_train, ari_train, X_val, ari_val, ari_val)

In [ ]:
# chosen_feature_names = ['log(n)/k * p_in/p_out', 'avg_sp']
chosen_feature_names = ['modularity', 'sbm_neighbour_score']
feature_indices = []
for feature in chosen_feature_names:
    feature_indices.append(feature_names.index(feature))
    print(feature, feature_names.index(feature))
print(feature_indices)

In [ ]:
trust_order = [k for k, v in sorted(list(zip(data_hub.kernel_names, ari_train.mean(axis=0))), key=lambda x: -x[1])]
trust_order

In [ ]:
estimator = OneVsRestClassifier(SVC(probability=True))
estimator.fit(X_train[:, feature_indices], y_train)
y_pred = estimator.predict_proba(X_val[:, feature_indices])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 18))
ax[0].imshow(np.mean(y_val.reshape((-1, 30, 14)), axis=1))
ax[0].set_xlabel('kernel')
ax[0].set_ylabel('SBM param column')
ax[0].set_title('ground truth')
ax[0].set_yticks(range(len(data_hub.datasets)))
ax[0].set_yticklabels(data_hub.datasets);
ax[0].set_xticks(range(len(suport_kernels_names)))
ax[0].set_xticklabels(suport_kernels_names, rotation=90);

ax[1].imshow(np.mean(y_pred.reshape((-1, 30, 14)), axis=1))
ax[1].set_xlabel('kernel')
ax[1].set_ylabel('SBM param column')
ax[1].set_title('prediction')
ax[1].set_yticks(range(len(data_hub.datasets)))
ax[1].set_yticklabels(data_hub.datasets);
ax[1].set_xticks(range(len(suport_kernels_names)))
ax[1].set_xticklabels(suport_kernels_names, rotation=90);

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 6), sharex=True, sharey=True)

x, y, v = [], [], []
for i in range(X_val.shape[0]):
    x.append(X_val[i, feature_indices[0]])
    y.append(X_val[i, feature_indices[1]])
    v.append(np.argmax(y_val[i]) if np.sum(y_val[i] == np.max(y_val[i])) == 1 else -1)
x, y, v = np.array(x), np.array(y), np.array(v)

for kernel_idx, kernel_name in enumerate(data_hub.kernel_names + ['several p=1']):
    if np.sum(v==kernel_idx) > 0:
        ax[0].scatter(x[v==kernel_idx], y[v==kernel_idx], label=kernel_name, color=d3_category20[kernel_idx], marker='+')
ax[0].set_xlabel(chosen_feature_names[0])
ax[0].set_ylabel(chosen_feature_names[1])
ax[0].legend()
ax[0].set_title('y_true')


x, y, v = [], [], []
for i in range(X_val.shape[0]):
    candidates = [k for k, v in zip(data_hub.kernel_names, y_pred[i] == np.max(y_pred[i])) if v]
#     if len(candidates) > 1:
#         print(candidates)
    for kernel in trust_order:
        if kernel in candidates:
            break
    kernel_idx = data_hub.kernel_names.index(kernel)
    
    x.append(X_val[i, feature_indices[0]])
    y.append(X_val[i, feature_indices[1]])
    v.append(kernel_idx)
x, y, v = np.array(x), np.array(y), np.array(v)
 
for kernel_idx, kernel_name in enumerate(data_hub.kernel_names + ['several p=1']):
    if np.sum(v==kernel_idx) > 0:
        ax[1].scatter(x[v==kernel_idx], y[v==kernel_idx], label=kernel_name, color=d3_category20[kernel_idx])
ax[1].set_xlabel(chosen_feature_names[0])
ax[1].set_ylabel(chosen_feature_names[1])
ax[1].legend()
ax[1].set_title('y_pred');


x, y, v = [], [], []
for xi in np.linspace(2, 6, 256):
    for yi in np.linspace(0.5, 5, 256):
        kernel_idx = np.argmax(estimator.predict_proba([[xi, yi]])[0])
        x.append(xi)
        y.append(yi)
        v.append(kernel_idx)
x, y, v = np.array(x), np.array(y), np.array(v)
for kernel_idx, kernel_name in enumerate(data_hub.kernel_names + ['several p=1']):
    if np.sum(v==kernel_idx) > 0:
        ax[2].scatter(x[v==kernel_idx], y[v==kernel_idx], label=kernel_name, color=d3_category20[kernel_idx])
ax[2].set_xlabel(chosen_feature_names[0])
ax[2].set_ylabel(chosen_feature_names[1])
ax[2].legend(loc=4)
ax[2].set_title('y_pred');
# ax[2].set_xlim(-0.5, 4.2)
# ax[2].set_ylim(0.5, 5)